In [1]:
from preprocesamiento import training_seq, SEQ_LEN
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

#es el tamaño del diccionario
OUTPUT_UNITS = 38
NUM_UNITS = 256
LOSS =  nn.CrossEntropyLoss(reduction = 'none')
LR = 0.01
EPOCH = 10
BATCH_SIZE = 32
SAVE_MODEL_PATH = 'model.pt'

#clase para generar una red lstm
class model_lstm(nn.Module):
    def __init__(self, output_units, num_units):
        super(model_lstm, self).__init__()
        self.lstm = nn.LSTM(output_units, num_units)
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(num_units, output_units)
        #agragamos soft max
        self.softmax = nn.Softmax(dim = 1)

    def feed_forward(self, x):
        output, _ = self.lstm(x)
        output = self.dropout(output)
        output = self.linear(output)
        #agregamos soft max
        output = self.softmax(output)

        return output

def train(output_units = OUTPUT_UNITS, num_units = NUM_UNITS, loss = LOSS, lr = LR):
    inputs, targets = training_seq(SEQ_LEN)
    targets = np.eye(38)[targets]

    #los pasamos a tensores
    inputs = torch.from_numpy(inputs).float()
    targets = torch.from_numpy(targets).long()

    #creamos la red y el optimizador
    red_lstm = model_lstm(output_units, num_units)
    optimizer = optim.Adam(red_lstm.parameters(), lr = lr)

    #entrenamos
    for epoch in range(EPOCH):
        loss_total = 0

        #mini batch
        for i in range(0, len(inputs), BATCH_SIZE):           
            batch_inputs = inputs[i:i+BATCH_SIZE]
            batch_targets = targets[i:i+BATCH_SIZE]
            
            print(f'batch_inputs = {batch_inputs.shape}')
            print(f'batch_targets = {batch_targets.shape}')

            optimizer.zero_grad()

            #hacemos feed forward
            outputs = red_lstm.feed_forward(batch_inputs)           
            print(f'outputs: {outputs.shape}')

            #tomamos la salida con mayor probabilidad
            # outputs = torch.argmax(outputs, dim = 1)
            print(f'outputs: {outputs.shape}')
            # loss_val = loss(outputs.permute(0, 1, 2), batch_targets)
            loss_val = loss(outputs, batch_targets)

            #backpropagation
            loss_val.sum().backward()
            optimizer.step()

            loss_total += loss_val.sum().item()

        print(f'Epoch: {epoch + 1} \t Loss: {loss_total:.5f}')

In [9]:
from preprocesamiento import training_seq, SEQ_LEN
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F

inputs, targets = training_seq(SEQ_LEN)
print(f'inputs: {inputs.shape}')
print(f'targets: {targets.shape}')

inputs: (308546, 32, 38)
targets: (308546,)


inputs: (308546, 32, 38)


targets: (308546,)

In [10]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [15]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        print(f'embeds: {embeds.shape}')
        print(type(embeds))
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        print(f'lstm_out: {lstm_out.shape}')
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        print(f'tag_space: {tag_space.shape}')
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [16]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

embeds: torch.Size([5, 6])
<class 'torch.Tensor'>
lstm_out: torch.Size([5, 1, 6])
tag_space: torch.Size([5, 3])
tensor([[-1.4400, -0.7945, -1.1671],
        [-1.3849, -0.7963, -1.2085],
        [-1.4646, -0.7948, -1.1484],
        [-1.2550, -0.7898, -1.3434],
        [-1.2265, -0.7861, -1.3821]])
embeds: torch.Size([5, 6])
<class 'torch.Tensor'>
lstm_out: torch.Size([5, 1, 6])
tag_space: torch.Size([5, 3])
embeds: torch.Size([4, 6])
<class 'torch.Tensor'>
lstm_out: torch.Size([4, 1, 6])
tag_space: torch.Size([4, 3])
embeds: torch.Size([5, 6])
<class 'torch.Tensor'>
lstm_out: torch.Size([5, 1, 6])
tag_space: torch.Size([5, 3])
embeds: torch.Size([4, 6])
<class 'torch.Tensor'>
lstm_out: torch.Size([4, 1, 6])
tag_space: torch.Size([4, 3])
embeds: torch.Size([5, 6])
<class 'torch.Tensor'>
lstm_out: torch.Size([5, 1, 6])
tag_space: torch.Size([5, 3])
embeds: torch.Size([4, 6])
<class 'torch.Tensor'>
lstm_out: torch.Size([4, 1, 6])
tag_space: torch.Size([4, 3])
embeds: torch.Size([5, 6])
<cl

In [18]:
a = torch.tensor([32, 38])
print(a.shape)
print(type(a))

torch.Size([2])
<class 'torch.Tensor'>
